# 非线性模型


In [48]:
import gurobipy as gp
import numpy as np
import math
import copy

In [49]:
def J_minus(j, bar_J):
    # j 之后的设施集合
    if j == 0:
        j_minus = [j]
    else:
        j_minus = copy.deepcopy(bar_J)
        j_minus.remove(j)
    return j_minus


def J_plus(j, bar_J):
    # j 之前的设施集合
    if j == 0:
        j_plus = copy.deepcopy(bar_J)
    else:
        j_plus = copy.deepcopy(bar_J)
        j_plus.remove(j)
        j_plus.remove(0)
    return j_plus

## 导入数据

In [50]:
# 导入数据
np.set_printoptions(suppress=True)    # 取消numpy打印的科学计数法


# cost 矩阵的第一索引位置是0 默认为虚拟设施
root = '../data/SnyderData/49nodes/'
cost = np.loadtxt(root+'cost.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

dmd = np.loadtxt(root+'dmd.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

fc = np.loadtxt(root+'fc.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

## 参数设置

In [51]:
rou = 0.05 # 损坏概率参数
max_visit_num = 5 # 客户的最大尝试次数

In [52]:
q = 0.05 * np.exp(-fc/200000) # 损坏概率
q[0] = 1

cus_num = len(dmd) # 客户数
node_num = cost.shape[0] # 节点数（虚拟 设施 客户）
fac_num = node_num - cus_num - 1 # 设施数

# 集合设置
J = [j for j in range(1, fac_num+1)] # 设施集合
I = [i for i in range(fac_num+1, node_num)] # 客户集合
bar_J = [j for j in range(0, fac_num+1)] # 设施拓展集合
R = [r for r in range(1,max_visit_num)] # 等级

# 常数集合
lmd = {i : dmd[i-fac_num-1] for i in I} # lambda需求
c = {(i,j) : cost[i,j] for i in bar_J+I for j in bar_J} # 价格
f = {j : fc[j] for j in J} # 建设成本


## 建模

In [53]:
m = gp.Model()

### 决策变量

In [54]:
x = m.addVars(((i,j,j_p,r) for i in I for j in bar_J for j_p in bar_J for r in R), vtype = gp.GRB.BINARY,name = 'x')
y = m.addVars((j for j in J), vtype = gp.GRB.BINARY, name = 'y')
z = m.addVars(((i,j) for i in I for j in bar_J), vtype = gp.GRB.BINARY, name = 'z')
p = m.addVars(((i,j,j_p,r) for i in I for j in bar_J for j_p in bar_J for r in R), lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS,name = 'p')

## 目标函数
$$
\min \sum_{j\in J}f_jy_j + \sum_{i\in I}\sum_{j\in \bar{J}} \lambda_i c_{ij} z_{ij}+\sum_{j\in \bar{J}}
\sum_{i\in I}\sum_{j'\in J_j^{-}} \sum_{r=1}^R \lambda_i c_{ijj'r} p_{ijj'r} x_{ijj'r}
$$

In [55]:
item_1 = gp.quicksum(f[j] * y[j] for j in J)
item_2 = gp.quicksum((lmd[i] * c[i,j] * z[i,j] ) for i in I for j in bar_J)
item_3 = gp.quicksum((lmd[i] * c[j,j_p] * p[i,j,j_p,r] * x[i,j,j_p,r]) for i in I for j in bar_J for j_p in J_minus(j,bar_J) for r in R )
m.setObjective(item_1 + item_2 + item_3)

print("done")

done


### 约束
$$
\sum_{j\in J} z_{ij} = 1,\forall i \in I
$$

In [56]:
m.addConstrs((gp.quicksum(z[i,j] for j in J)==1 for i in I), name='Primary')
print("done")

done


$$
z_{ij} + \sum_{j'\in J_j^+}\sum_{r=1}^{R}x_{ij'jr} 
\le y_j ,\quad \forall i\in I,j\in J
$$

In [57]:
m.addConstrs((z[i,j] + gp.quicksum(x[i,j_p,j,r] for r in R for j_p in J_plus(j, bar_J)) <= y[j] for i in I for j in J), name='assign2open')
print("done")

done


$$
z_{ij} = \sum_{j'\in J_j^-} x_{ijj'1}, \forall i \in I, j\in \bar{J} \tag{13}
$$

In [58]:
m.addConstrs((z[i,j] == gp.quicksum(x[i,j,j_p,1] for j_p in J_minus(j, bar_J)) for i in I for j in bar_J), name='rankPrimary')
print("done")

done


$$
\sum_{j'\in J_j^+}x_{i,j',j,r-1} = \sum_{j'\in J_j^-}x_{i,j,j',r},\forall i\in I,j\in \bar{J},r = 2,...,R
$$

In [59]:
m.addConstrs(((gp.quicksum(x[i,j_p,j,r-1] for j_p in J_plus(j,bar_J)) 
            == gp.quicksum(x[i,j,j_p,r] for j_p in J_minus(j,bar_J))) for i in I for j in bar_J for r in R[1:]), name='rank')
print("done")

done


$$
\sum_{j\in \bar{J}} x_{ijj_0R} = 1,\forall i \in I \tag{15}
$$

In [60]:
m.addConstrs(((gp.quicksum(x[i,j,0,R[-1]] for j in J)) == 1 for i in I), name='rankLast')
print("done")

done


$$
p_{ijj'1} = q_j,\forall i\in I,j \in \bar{J}, j'\in J_j^-
$$

In [61]:
m.addConstrs((p[i,j,j_p,1] == q[j] for i in I for j in bar_J for j_p in J_minus(j,bar_J)), name='probPrimary')
print("done")

done


$$
p_{ijj'r} = q_j \sum_{j'\in J_j^+}p_{ij'j(r-1)}x_{ij'j(r-1)},\forall i\in I,j\in \bar{J},j'\in J_j^-,r=2,...,R
$$

In [62]:
m.addConstrs(((p[i,j,j_p,r] == q[j]*gp.quicksum(p[i,j_m,j,r-1] * x[i,j_m,j,r-1]for j_m in J_plus(j,bar_J))) for i in I for j in bar_J for j_p in J_minus(j, bar_J) for r in R[1:]), name='probPrimary')
print("done")

done


## 求解

In [63]:
m.Params.MIPGap = 0.00001
m.Params.timeLimit = 1000

# m.Params.LogFile =  "SolvingLog.log"
# m.write('Model.lp')

m.optimize()

# m.computeIIS()
# m.write('Model.ilp')
# m.write('Model.lp')

m.write('Solution.sol')

print('求解完成')

Set parameter MIPGap to value 1e-05
Set parameter TimeLimit to value 1000
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 129997 rows, 982499 columns and 1414630 nonzeros
Model fingerprint: 0x50ddfbb9
Model has 470596 quadratic objective terms
Model has 353094 quadratic constraints
Variable types: 490000 continuous, 492499 integer (492499 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e-02, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+02, 3e+06]
  QObjective range [3e+02, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 1e+00]
Presolve removed 117747 rows and 38514 columns (presolve time = 5s) ...
Presolve removed 121943 rows and 160408 columns (presolve time = 10s) ...
Presolve added 226251 rows and 0 columns
Presolve removed 0 rows and 160740 columns
Presolve time: 14.37s
Presolved: 1184985 rows, 11